# Preparation

Imports libraries

Loads RedMetrics data

In [ ]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import seaborn as sns
import re
import json
import datetime

from random import randint
from ipywidgets import FloatProgress,IntProgress,IntText,Text,interact,interactive,IntSlider,FloatSlider
from IPython.display import display
from itertools import chain

In [ ]:
pd.__version__

In [ ]:
np.__version__

In [ ]:
processRMDF1522 = not ('rmdf1522' in globals())
processRMDF160 = not ('rmdf160' in globals())
processRMDFTest = not ('rmrdftest' in globals())

### common variables

In [ ]:
last1522DataFilesNamesStem = "2018-04-27"
dataFilesNamesStem = "2018-05-23"

dataFolderPath = "../../data/"
version1522Suffix = '-1.52.2'
version160Suffix = '-1.60'
processedSuffix = "-processed"
testUsersSuffix = "-testUsers"
testSuffix = ".test"
csvEncoding = 'utf-8'
csvSuffix = '.csv'

checkpointsStem = "tutorial1.Checkpoint"

In [ ]:
processedRMDF1522Path = dataFolderPath + last1522DataFilesNamesStem + version1522Suffix + processedSuffix + csvSuffix
processedRMDF160Path  = dataFolderPath + dataFilesNamesStem         + version160Suffix  + processedSuffix + csvSuffix

rmrdf1522Path         = dataFolderPath + last1522DataFilesNamesStem + version1522Suffix                   + csvSuffix
rmrdf160Path          = dataFolderPath + dataFilesNamesStem         + version160Suffix                    + csvSuffix

rmrdfTestPath         = dataFolderPath + dataFilesNamesStem         + testSuffix                          + csvSuffix
testUsersPath         = dataFolderPath                              + testUsersSuffix                     + csvSuffix

In [ ]:
rmdfLoadColumnNames = ['id', 'serverTime', 'userTime',\
                   'playerId', 'playerCustomData',\
                   'type', 'coordinates','section',\
                   'customData.biobrick', 'customData.devices',\
                   'customData.slot', 'customData.sound',\
                   'customData','customData.duration',\
                   'customData.nanobot', 'customData.language',\
                   'customData.controls', 'customData.chapter',\
                   'customData.life','customData.source',\
                   'customData.platform','customData.localplayerguid',\
                   'customData.sametab', 'customData.device',\
                   'customData.energy', 'customData.option',\
                   'customData.newtab','customData.dnabit',\
                   'customData.count', 'customData.plasmid',\
                   'customData.total', 'customData.message',\
                   'customData.graphics']

In [ ]:
### columns
# In RedMetrics data, 'playerId' is actually a session ID.
# Permanent player IDs are stored as 'localplayerguid' in 'customdata' attached to 'start' events.
rmdfMinimalInitializationColumns = ['customData.localplayerguid']
rmdfInitializationColumns = ['customData.localplayerguid', 'playerId', 'type', 'serverTime', 'customData.platform']
# TODO check use
# rmdfRelevantColumns = ['sessionId', 'serverTime', 'userId', 'customData.platform']
rmdfPlayerFilteringColumns = ['sessionId', 'userId', 'customData.platform', 'serverTime']
rmdfCheckpointsRelevantColumns = ['sessionId', 'userId', 'type', 'section']
rmdfRelevantColumns = ['customData.localplayerguid', 'playerId', 'type']
rmdfRenamedRelevantColumns = ['userId', 'sessionId', 'type']

# Functions

In [ ]:
## Loading
### Data format fixes
def userIdConverter(uId):
    sUID = str(uId)
    if(sUID.startswith('n')):# == 'nan' or == 'null'):
        return ''
    else:
        return sUID.replace('"','')

# date string to pd.Timestamp
#  RedMetrics timestamps are always UTC according to doc
#  https://github.com/CyberCRI/RedMetrics/blob/master/API.md
rmdfDateparse = lambda x: pd.to_datetime(x, utc=True)

def safeGetNormalizedRedMetricsCSV( df ):
    return df.rename(index=str, columns={'customData.localplayerguid' : 'userId', 'playerId': 'sessionId'})
    
def getNormalizedRedMetricsCSV( df ):
    newColumns = np.unique(np.concatenate((rmdfMinimalInitializationColumns, df.columns.values)))
    return safeGetNormalizedRedMetricsCSV(df.loc[:,newColumns])

def writeTestUsers(testUsers):
    testUsers.to_csv(testUsersPath, encoding=csvEncoding)

def writeRMDF(rmdf, rmdfPath):
    rmdf.to_csv(rmdfPath, encoding=csvEncoding)

## Filtering

In [ ]:
def getAllSessions( _rmDF, dropna ):
    _result = _rmDF.loc[:, rmdfRenamedRelevantColumns]
    _result = _result[_result['type']=='start']
    _result = _result.drop('type', 1)
    if dropna:
        _result = _result.dropna(how='any')
    return _result

def getTestSessions(_rmDF, _rmTestDF, includeAndroid = True, includeEditor = True, includeTest = True):

    rmDFTestSessions = set()
    rmTestDFTestSessions = set()

    #  - have 'android' or '...editor' as platform
    if(includeAndroid):
        rmDFTestSessions |= set(_rmDF[_rmDF['customData.platform'].isin(['"android"'])]['sessionId'])
    if(includeEditor):
        rmDFTestSessions |= set(_rmDF[_rmDF['customData.platform'].apply(lambda s: str(s).endswith('editor"'))]['sessionId'])
    #print(str(len(rmDFTestSessions)))

    #  - are in the RedMetrics test channel
    if(includeTest):
        rmTestDFTestSessions = set(_rmTestDF['sessionId'])
    #print(str(len(rmTestDFTestSessions)))

    #  - belong to a user who has a session of the type above
    # all the sessions above
    testSessions = rmDFTestSessions | rmTestDFTestSessions

    return testSessions

# gets sessions which either:
#  - have 'android' or '...editor' as platform
#  - are in the RedMetrics test channel
#  - belong to a user who has a session of the type above
def getTestUsersSessions(_rmDF, _rmTestDF, includeAndroid = True, includeEditor = True, includeTest = True):

    # tables of association of user-sessions
    rmDFUsersSessions = getAllSessions(_rmDF, False)
    rmTestDFUsersSessions = getAllSessions(_rmTestDF, False)
    userSessions = pd.concat([rmDFUsersSessions,rmTestDFUsersSessions])

    testSessions = getTestSessions(_rmDF, _rmTestDF,
                                   includeAndroid = includeAndroid, includeEditor = includeEditor, includeTest = includeTest)

    # all the users
    rmDFTestUsers = set(rmDFUsersSessions[rmDFUsersSessions['sessionId'].isin(testSessions)]['userId'].dropna())
    rmTestDFTestUsers = set(_rmTestDF['userId'].dropna())
    rmTestDFTestUsers.remove('')
    testUsers = rmDFTestUsers | rmTestDFTestUsers
    # all the sessions which belong to these users
    allTestSessions = set(rmDFUsersSessions[rmDFUsersSessions['userId'].isin(testUsers)]['sessionId'].dropna())

    return (testUsers,allTestSessions)

## Load and process

In [ ]:
# if the processing of the rmrdfs has already been done,
# just load the preprocessed rmdfs
#if processRMDF1522:
def loadProcessedRMDFs():
    ## Try loading the pre-processed dataframe
    rmdfTestUsers = set()
    rmdf1522 = []
    rmdf160 = []
    try:

        rmdfTestUsers = set(pd.read_csv(testUsersPath, dtype=str)['userId'])
        print("rmdfTestUsers read_csv success (1/3)")
        
        rmdf1522 = pd.read_csv(\
                            processedRMDF1522Path,\
                            dtype=str, parse_dates=['serverTime','userTime'],\
                            date_parser=rmdfDateparse,\
                           )
        if rmdf1522.columns[0] == 'Unnamed: 0':
            rmdf1522 = rmdf1522.iloc[:,1:]
        print("rmdf1522 read_csv success (2/3)")

        rmdf160 = pd.read_csv(\
                            processedRMDF160Path,\
                            dtype=str, parse_dates=['serverTime','userTime'],\
                            date_parser=rmdfDateparse,\
                           )
        if rmdf160.columns[0] == 'Unnamed: 0':
            rmdf160 = rmdf160.iloc[:,1:]
        print("rmdf160 read_csv success (3/3)")

    except FileNotFoundError:
        print("rmdfs will be loaded, processed, saved")

    return (rmdfTestUsers, rmdf1522, rmdf160)


### RMDFTest loading
# necessary variables for RMDFTest loading:
# dataFolderPath
# dataFilesNamesStem
# dateparse
# userIdConverter
# rmdfLoadColumnNames
# getNormalizedRedMetricsCSV

# raw redmetrics df loading
def loadRMRDF(rmdfPath):
    rmrdf  = pd.read_csv(\
                            rmdfPath,\
                            dtype=str,\
                            parse_dates=['serverTime','userTime'],\
                            date_parser=rmdfDateparse,\
                            converters={'customData.localplayerguid':userIdConverter},\
                         )
    rmrdf = rmrdf.loc[:,rmdfLoadColumnNames]
    normalizedRMDF = getNormalizedRedMetricsCSV(rmrdf)
    return normalizedRMDF

# processing of raw redmetrics dfs
# rmdfTestUsers is a set
# rmdf1522 is assumed to be set
# rmrdfPath raw df path for reading
# rmdfPath processed df path for writing
def processRMDF(rmrdfPath, rmdfPath, normalizedRMDFTest, rmdfTestUsers):
    #print("processRMDF start")
    normalizedRMDF = loadRMRDF(rmrdfPath)

    #print("call to getTestUsersSessions...")
    (testUsers, allTestSessions) = getTestUsersSessions(_rmDF = normalizedRMDF, _rmTestDF = normalizedRMDFTest)
    #print("call to getTestUsersSessions done")
    rmdf = normalizedRMDF[~normalizedRMDF['sessionId'].isin(allTestSessions)]
    
    # check that testUsers is a subset of the known rmdfTestUsers
    ## print("#testUsers="+str(len(testUsers)))
    #if "userId" in testUsers:
        #print("userId in testUsers")
    #if "userId" in rmdfTestUsers:
        #print("userId in rmdfTestUsers")
        
    #print("subset test")
    #print(type(testUsers))
    #print(type(rmdfTestUsers))
    if not testUsers.issubset(rmdfTestUsers):
        diff = testUsers.difference(rmdfTestUsers)
        #print("found " + str(len(diff)) + " unknown test users in rmdf " + str(diff))
#########################################################################################################
        testUsers = testUsers | rmdfTestUsers
#########################################################################################################
        testUsers = pd.DataFrame(data=list(testUsers), columns=['userId'])
        writeTestUsers(testUsers)

    #print("userSessions")
    userSessions = rmdf[rmdf['userId']!=''].loc[:,['userId','sessionId']].dropna(how='any').drop_duplicates()
    
    intProgress = IntProgress(min=0, max=len(userSessions.index))
    display(intProgress)
    intText = IntText(0)
    display(intText)
    
    #print("loop starting")
    for userSessionsIndex in userSessions.index:
        intProgress.value += 1
        intText.value += 1
        
        userId = userSessions.loc[userSessionsIndex, 'userId']
        sessionId = userSessions.loc[userSessionsIndex, 'sessionId']
        rmdf.loc[rmdf['sessionId']==sessionId,'userId'] = userId

    #rmdf1522['userId'].nunique(),userSessions['userId'].nunique(),\
    #rmdf1522[~rmdf1522['userId'].isin(userSessions['userId'].unique())],\
    #userSessions[~userSessions['userId'].isin(rmdf1522['userId'].unique())]

#### Saving to csv
    #print("saving to csv")
    writeRMDF(rmdf, rmdfPath)
    #print("processRMDF done")
    
    return (rmdf, testUsers)

# Execution

In [ ]:
processRMDF1522 = not ('rmdf1522' in globals())
processRMDF160 = not ('rmdf160' in globals())
processRMDFTest = not ('normalizedRMDFTest' in globals())

In [ ]:
processRMDF1522, processRMDF160, processRMDFTest

In [ ]:
if processRMDF1522 or processRMDF160 or processRMDFTest:
    
    ## calls
    #print("STEP 1")
    (rmdfTestUsers, rmdf1522, rmdf160) = loadProcessedRMDFs()
    #print(type(rmdfTestUsers))

    process1522 = (len(rmdf1522) == 0)
    process160  = (len(rmdf160)  == 0)

    normalizedRMDFTest = []
    testUsers = []

    if process1522 or process160:
        #print("STEP test")
        normalizedRMDFTest = loadRMRDF(rmrdfTestPath)
    if process1522:
        #print("STEP 1522")
        (rmdf1522, testUsers) = processRMDF(rmrdf1522Path, processedRMDF1522Path, normalizedRMDFTest, rmdfTestUsers)
    if process160:
        #print("STEP 160")
        (rmdf160, testUsers) = processRMDF(rmrdf160Path, processedRMDF160Path, normalizedRMDFTest, rmdfTestUsers)
    
    # concatenation of all redmetrics events pertaining to 1.52.2 survey: rmdf1522 and rmdf160
    rmdfConcat = pd.concat([rmdf1522, rmdf160])
    rmdfConcat.index = range(0, len(rmdfConcat.index))
    
    #print("STEP done")
#else:
    #print("all done")

In [ ]:
#rmdf1522['userId'].nunique(), rmdf160['userId'].nunique()

In [ ]:
#rmdf1522['userTime'].min()

# All versions

rdf = pd.concat([part100, 
                      part131, part132, part133, 
                      part140, 
                      part150, part151, part1522])

df = getNormalizedRedMetricsCSV(rdf)

# Old versions

rdf100 = pd.read_csv("../../data/1.0.csv")
rdf131 = pd.read_csv("../../data/1.31.csv")
rdf132 = pd.read_csv("../../data/1.32.csv")
rdf133 = pd.read_csv("../../data/1.33.csv")
rdf140 = pd.read_csv("../../data/1.40.csv")
rdf150 = pd.read_csv("../../data/1.50.csv")
rdf151 = pd.read_csv("../../data/1.51.csv")

part100 = rdf100.loc[:,relevantColumns]
part131 = rdf131.loc[:,relevantColumns]
part132 = rdf132.loc[:,relevantColumns]
part133 = rdf133.loc[:,relevantColumns]
part140 = rdf140.loc[:,relevantColumns]
part150 = rdf150.loc[:,relevantColumns]
part151 = rdf151.loc[:,relevantColumns]

# Tests

rdftest = pd.read_csv("../../data/2017-10-11.test.csv")
dftest = getNormalizedRedMetricsCSV(rdftest)

TOD: get rid of warning
    
    DtypeWarning: Columns (18,22,28,32,38) have mixed types. Specify dtype option on import or set low_memory=False.
    interactivity=interactivity, compiler=compiler, result=result)

using https://stackoverflow.com/questions/24251219/pandas-read-csv-low-memory-and-dtype-options